# Weekly Project 5 
## Implementation of global registration 
### Task 1
Today, your task is to implement a global registration algorithm.

It should be able to roughly align two point clouds.
Implement the global registration, and then try the following:

1. Can you fit `r1.pcd` and `r2.pcd`?
2. Can you fit `car1.ply` and `car2.ply`?
The corresponding files are in the `global_registration` folder.


### Task 2 (Challange)
Challanges attempt either or both:
- Implement local registration.

- Attempt to reconstruct the car from the images in `car_challange` folder.

You can use the notebooks from Monday as a starting point.

In [2]:
import open3d as o3d
import numpy as np
import copy
import os

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
# helper function for drawing
# If you want it to be more clear set recolor=True
def draw_registrations(source, target, transformation = None, recolor = False):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    if(recolor):
        source_temp.paint_uniform_color([1, 0.706, 0])
        target_temp.paint_uniform_color([0, 0.651, 0.929])
    if(transformation is not None):
        source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp])

In [3]:
# GLOBAL REGISTRATION
# read pointcloud
source = o3d.io.read_point_cloud("global_registration/r1.pcd")
target = o3d.io.read_point_cloud("global_registration/r2.pcd")

# Used for downsampling.
voxel_size = 0.05
source = source.voxel_down_sample(voxel_size=voxel_size)
target = target.voxel_down_sample(voxel_size=voxel_size)
source.estimate_normals()
target.estimate_normals()

# Show models side by side
draw_registrations(source, target)

In [4]:
# global registration with features
def global_registration(source, target, vs, tr, rf):
    point_to_point =  o3d.pipelines.registration.TransformationEstimationPointToPoint(False)
    radius_feature = vs * rf
    f_source = o3d.pipelines.registration.compute_fpfh_feature(source, o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=1000))
    f_target = o3d.pipelines.registration.compute_fpfh_feature(target, o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=1000))
    distance_threshold = vs * tr

    corr_length = 0.9


    c0 = o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(corr_length)
    c1 = o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(distance_threshold)
    c2 = o3d.pipelines.registration.CorrespondenceCheckerBasedOnNormal(0.095)

    checker_list = [c0, c1, c2]

    ransac_result = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
        source, target, 
        f_source, f_target, True,
        distance_threshold,
        point_to_point,
        checkers = checker_list,)
        # criteria=o3d.pipelines.registration.RANSACConvergenceCriteria(10000000, 0.999))
    evaluation = o3d.pipelines.registration.evaluate_registration(source, target, tr, ransac_result.transformation) 
    return ransac_result, evaluation.fitness

In [5]:
ransac_result = global_registration(source, target, voxel_size, 1.5, 5)
draw_registrations(source, target, ransac_result.transformation, True)


In [6]:
# NOW WITH THE CAR DATA
source = o3d.io.read_point_cloud("global_registration/car1.ply")
target = o3d.io.read_point_cloud("global_registration/car2.ply")

# Used for downsampling.
voxel_size = 0.05
source = source.voxel_down_sample(voxel_size=voxel_size)
target = target.voxel_down_sample(voxel_size=voxel_size)

source.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=0.5,
                        max_nn=30),fast_normal_computation=True)
target.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=0.5,
                        max_nn=30),fast_normal_computation=True)

draw_registrations(source, target, None, True)
ransac_result = global_registration(source, target, voxel_size,0.9, 2)
draw_registrations(source, target, ransac_result.transformation, True)

CHALLENGE TIME

In [5]:
# Implement local registration

def local_registration(source, target, threshold):
    trans_init = np.asarray([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]])
          
    point_to_plane =  o3d.pipelines.registration.TransformationEstimationPointToPlane()
    icp_result = o3d.pipelines.registration.registration_icp(
        source, target, threshold, trans_init,
        point_to_plane)
    evaluation = o3d.pipelines.registration.evaluate_registration(source, target, threshold, icp_result.transformation)
    return icp_result, evaluation.fitness

    

In [22]:
# read pointcloud
source = o3d.io.read_point_cloud("global_registration/r1.pcd")
target = o3d.io.read_point_cloud("global_registration/r2.pcd")
# Used for downsampling.
voxel_size = 0.05
source = source.voxel_down_sample(voxel_size=voxel_size)
target = target.voxel_down_sample(voxel_size=voxel_size)
# source.estimate_normals()
# target.estimate_normals()
source.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=0.5,
                        max_nn=30),fast_normal_computation=True)
target.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=0.5,
                        max_nn=30),fast_normal_computation=True)

# Show models side by side
draw_registrations(source, target)

# START WITH GLOBAL REGISTRATION 
ransac_result = global_registration(source, target, voxel_size,0.9, 2)
draw_registrations(source, target, ransac_result.transformation, True)
source = source.transform(ransac_result.transformation)
# DO LOCAL REGISTRATION
threshold = 0.3
res = local_registration(source, target, threshold)
draw_registrations(source, target, res.transformation, True)

RegistrationResult with fitness=7.420168e-01, inlier_rmse=6.027961e-02, and correspondence_set size of 3532
Access transformation to get result.


In [ ]:
# Attempt to reconstruct the car from the images in car_challange folder.
# idea: read the images, match in batches (global plus local alignment), then downsample 

In [6]:
def getPointCloud(path_color, path_depth, camera, T, voxel_size):
    color = o3d.io.read_image(path_color)
    depth = o3d.io.read_image(path_depth)
    rgbd_image = o3d.geometry.RGBDImage.create_from_color_and_depth(
        color, 
        depth, 
        convert_rgb_to_intensity = False)
    pc = o3d.geometry.PointCloud.create_from_rgbd_image(
        rgbd_image, camera)
    pc.transform(T)
    pc = pc.voxel_down_sample(voxel_size=voxel_size)
    pc.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=0.5,
                        max_nn=30),fast_normal_computation=True)
    return pc

In [28]:
# recreate object pointcloud
camera = o3d.camera.PinholeCameraIntrinsic(
        o3d.camera.PinholeCameraIntrinsicParameters.PrimeSenseDefault)
T = [[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]]

## PARAMETERS
voxel_size = 0.5
batch_size = 10
jumps = 1
threshold = 0.2
trans_init = np.asarray([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]])

## READ FILES
color_dir = 'car_challange/rgb/'
depth_dir = 'car_challange/depth/'
color_files = sorted(os.listdir(color_dir))
depth_files = sorted(os.listdir(depth_dir))

# read first PC 
path_color = os.path.join(color_dir, color_files[0])
path_depth = os.path.join(depth_dir, depth_files[0])
match_pc = getPointCloud(path_color, path_depth, camera, T, voxel_size)
pc_prev = copy.deepcopy(match_pc)

images = []     # RGB images
pcs = []        # pointclouds 
indx = 1

while indx < len(color_files):
    # match PC with prev PC and then add matches onto each other  
    path_color = os.path.join(color_dir, color_files[indx])
    path_depth = os.path.join(depth_dir, depth_files[indx])
    pc = getPointCloud(path_color, path_depth, camera, T, voxel_size)

    ########### HERE THE open3d MAGIC HAPPENS ####################

    # do global registration 
    ransac_result, fit_glob = global_registration(pc, pc_prev, voxel_size, 1.3 ,2) # 0.9, 2
    if fit_glob > 0.6:
        # transform 
        pc.transform(ransac_result.transformation)
        # do local registration 
        ransac_result, fit_loc = local_registration(pc, pc_prev, 0.3)
        # downsample every x batches 
        if fit_loc > 0.7:
            pc.transform(ransac_result.transformation)
            # match_pc = match_pc.voxel_down_sample(voxel_size=voxel_size)
            # add matches but match with prev not total 
            pc_prev = pc
            # pc_prev.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=0.5,
            #                 max_nn=30),fast_normal_computation=True)
            match_pc += pc
            # if(indx % ((jumps+1) * batch_size) == 0):
            #     # o3d.visualization.draw_geometries([match_pc])
            #     match_pc = match_pc.voxel_down_sample(voxel_size=voxel_size)
            #     # o3d.visualization.draw_geometries([match_pc])
            #     print("--Downsampling--")
                
            print("Match " + str(indx) + " of " + str(1722) + ' fitting glob: ' + str(fit_glob) + ' loc: ' + str(fit_loc))
        else:
            print("--Discard Local--" + ' fitting: ' + str(fit_loc))
    else:
        print("--Discard Global--" + ' fitting: ' + str(fit_glob))
        
    indx += jumps

    ##############################################################


    # o3d.visualization.draw_geometries([pc])
    # o3d.visualization.draw_geometries([match_pc])

Match 1 of 1722 fitting glob: 1.0 loc: 1.0
Match 2 of 1722 fitting glob: 1.0 loc: 1.0
Match 3 of 1722 fitting glob: 1.0 loc: 1.0
Match 4 of 1722 fitting glob: 1.0 loc: 0.9827586206896551
Match 5 of 1722 fitting glob: 1.0 loc: 1.0
Match 6 of 1722 fitting glob: 1.0 loc: 1.0
Match 7 of 1722 fitting glob: 1.0 loc: 1.0
Match 8 of 1722 fitting glob: 1.0 loc: 1.0
Match 9 of 1722 fitting glob: 1.0 loc: 1.0
Match 10 of 1722 fitting glob: 1.0 loc: 1.0
Match 11 of 1722 fitting glob: 1.0 loc: 1.0
Match 12 of 1722 fitting glob: 1.0 loc: 1.0
Match 13 of 1722 fitting glob: 1.0 loc: 0.9827586206896551
Match 14 of 1722 fitting glob: 1.0 loc: 1.0
Match 15 of 1722 fitting glob: 1.0 loc: 1.0
Match 16 of 1722 fitting glob: 1.0 loc: 1.0
Match 17 of 1722 fitting glob: 1.0 loc: 0.9836065573770492
Match 18 of 1722 fitting glob: 1.0 loc: 1.0
Match 19 of 1722 fitting glob: 1.0 loc: 0.9672131147540983
Match 20 of 1722 fitting glob: 1.0 loc: 1.0
Match 21 of 1722 fitting glob: 1.0 loc: 0.9833333333333333
Match 22 o

KeyboardInterrupt: 

In [27]:
o3d.visualization.draw_geometries([match_pc])